In [106]:
import pandas as pd
xmas = pd.read_csv("/Users/avanti/OneDrive/MASTERS/GSB544-Computing and Machine Learning/Labs/Lab Data/xmas.csv")

In [ ]:
xmas

function 1: pluralize_gift()

In [108]:
def pluralize_gift(gift):
    """
    Returns plural of a noun

    Parameters
    ----------
    gift: str
        A noun

    Return
    ------
    str
        Plural version
    """

    # Dictionary for irregular plural forms
    irregulars = {
        "goose": "geese"
    }

    # Check for irregular forms first
    if gift in irregulars:
        return irregulars[gift]
    
    # initially written by ChatGPT
    if gift.endswith('y') and not gift.endswith('ay'):
        return gift[:-1] + 'ies'  # Converts 'y' to 'ies'
    elif (gift.endswith('s') or gift.endswith('x') or gift.endswith('ch')) and not gift.endswith('es') and not gift.endswith('ds'):
        return gift + 'es'  # Adds 'es' for special plural cases
    elif not gift.endswith('ds'):
        return gift + 's'  # Regular plural case

    return gift



In [109]:
# Should work
print(pluralize_gift("goose"))

# pluralizes all gifts in gift column without vectorizing the function
xmas['plural_gift'] = xmas['Gift.Item'].apply(pluralize_gift)

geese
geese


function 2: make_phrase()

In [110]:
# initially written by ChatGPT
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Returns a phrase for the '12 Days of Christmas' song.

    Parameters
    ----------
    num: int
        The day number
    num_word: str
        The number as a word (e.g., "ten")
    item: str
        The gift item (e.g., "lords")
    verb: str
        The action (e.g., "a-leaping")
    adjective: str
        Optional adjective (e.g., "golden")
    location: str
        Optional location (e.g., "in a pear tree")

    Returns
    -------
    str
        The formatted phrase
    """

    ## Step 1: Replace NAs with blank strings
    verb = verb if pd.notna(verb) else ""
    adjective = adjective if pd.notna(adjective) else ""
    location = location if pd.notna(location) else ""

    ## Step 2: If the day number is larger than 1, pluralize the item
    if num > 1:
        item = pluralize_gift(item).strip()

    ## Step 3: Check if the item starts with a vowel
    vowels = ('a', 'e', 'i', 'o', 'u')
    starts_with_vowel = item[0].lower() in vowels

    ## Step 4: Determine the correct article for the first day
    if num == 1:
        num_word = "an" if starts_with_vowel else "a"

    ## Step 5: Construct the phrase
    phrase = f"{num_word} {adjective} {item} {verb} {location}".strip()

    return phrase

In [111]:
# dictionary to map numbers to words
number_to_word = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
}

# initially written by ChatGPT
# Apply the dictionary to map 'Day' column to 'Day.in.Words'
xmas['Day.in.Words'] = xmas['Day'].map(number_to_word)

# Check the updated dataframe
# print(xmas[['Day', 'Day.in.Words']])

In [112]:
# testing make_phrase function
make_phrase(num=10, num_word="ten", item="lords", verb="a-leaping", adjective="", location="")
# Output: "ten lords a-leaping"

'ten  lords a-leaping'

In [113]:
# initially written by ChatGPT
# Builds the full phrases with the pluralized gifts, day, location, verb, adjective
xmas['Full.Phrase'] = xmas.apply(
    lambda row: make_phrase(row['Day'], row['Day.in.Words'], row['Gift.Item'], row['Verb'], row['Adjective'], row['Location']).strip(),
    axis=1)

In [114]:
print(xmas[['Day', 'Full.Phrase']])

    Day                   Full.Phrase
0     1  a  partridge  in a pear tree
1     2              two turtle doves
2     3             three french hens
3     4            four calling birds
4     5             five golden rings
5     6           six  geese a-laying
6     7       seven  swans a-swimming
7     8        eight  maids a-milking
8     9          nine  ladies dancing
9    10          ten  lords a-leaping
10   11         eleven  pipers piping
11   12     twelve  drummers drumming
    Day                   Full.Phrase
0     1  a  partridge  in a pear tree
1     2              two turtle doves
2     3             three french hens
3     4            four calling birds
4     5             five golden rings
5     6           six  geese a-laying
6     7       seven  swans a-swimming
7     8        eight  maids a-milking
8     9          nine  ladies dancing
9    10          ten  lords a-leaping
10   11         eleven  pipers piping
11   12     twelve  drummers drumming


function 3: sing_day()

In [115]:
# define a function to convert numbers to words but for the intro
def number_to_word2(num):
    words = {
        1: "first",
        2: "second",
        3: "third",
        4: "fourth",
        5: "fifth",
        6: "sixth",
        7: "seventh",
        8: "eighth",
        9: "ninth",
        10: "tenth",
        11: "eleventh",
        12: "twelfth",
    }
    return words.get(num, str(num))  # Fallback to string if not found

In [116]:
# initially written by ChatGPT
def sing_day(dataset, num, phrase_col):
    """
    Generate the lyrics for a specific day of the Christmas song.

    Parameters
    ----------
    dataset: DataFrame
        The dataset containing the gift phrases.
    num: int
        The day of Christmas to sing about.
    phrase_col: str
        The name of the column containing the gift phrases.

    Returns
    -------
    str
        The lyrics for the specified day.
    """

    # Step 1: Setup the intro line
    num_word = number_to_word2(num)  # convert "1" to "first" etc.
    intro = f"On the {num_word} day of Christmas, my true love gave to me:"

    # Step 2: Sing the gift phrases
    gifts = ""
    for i in range(num, 0, -1):  # Gifts sung in reverse order
        gift_phrase = dataset.loc[i - 1, phrase_col].strip()  # Clean the gift phrase
        gifts += gift_phrase + (",\n" if i > 1 else "")  # Add comma for all but last gift

    # Add "and" before the last gift if there's more than one gift
    if num > 1:
        gifts = gifts.rsplit(',', 1)  # Split to modify last gift
        gifts = gifts[0].strip() + ", and\n" + gifts[1].strip()  # Add comma before "and" for last gift

    # Step 3: Put it all together and return
    return f"{intro}\n{gifts.strip()}."  # Clean up spaces and add period

In [117]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves, and
a  partridge  in a pear tree.
On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves, and
a  partridge  in a pear tree.


entire song

In [118]:
# initially written by ChatGPT
def sing_entire_song(dataset, phrase_col):
    """
    Generate and print the lyrics for the entire 12 Days of Christmas song.

    Parameters
    ----------
    dataset: DataFrame
        The dataset containing the gift phrases.
    phrase_col: str
        The name of the column containing the gift phrases.
    """
    # Create a Series of days from 1 to 12
    days = pd.Series(range(1, 13))

    # Use .map() to get lyrics for each day and print them
    lyrics = days.map(lambda day: sing_day(dataset, day, phrase_col))
    # Join with an extra newline to create space between chunks
    print("\n\n".join(lyric.strip() for lyric in lyrics))  # Strip each lyric before joining

In [119]:
sing_entire_song(xmas, 'Full.Phrase')

On the first day of Christmas, my true love gave to me:
a  partridge  in a pear tree.

On the second day of Christmas, my true love gave to me:
two turtle doves, and
a  partridge  in a pear tree.

On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves, and
a  partridge  in a pear tree.

On the fourth day of Christmas, my true love gave to me:
four calling birds,
three french hens,
two turtle doves, and
a  partridge  in a pear tree.

On the fifth day of Christmas, my true love gave to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a  partridge  in a pear tree.

On the sixth day of Christmas, my true love gave to me:
six  geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a  partridge  in a pear tree.

On the seventh day of Christmas, my true love gave to me:
seven  swans a-swimming,
six  geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtl

surprise song

In [120]:
xmas2 = pd.read_csv("/Users/avanti/OneDrive/MASTERS/GSB544-Computing and Machine Learning/Labs/Lab Data/xmas_2.csv")

In [121]:
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


In [122]:
xmas2['plural_gift'] = xmas2['Gift.Item'].apply(pluralize_gift)
xmas2['Day.in.Words'] = xmas2['Day'].map(number_to_word)
xmas2['Full.Phrase'] = xmas2.apply(
    lambda row: make_phrase(row['Day'], row['Day.in.Words'], row['Gift.Item'], row['Verb'], row['Adjective'], row['Location']),
    axis=1)
sing_entire_song(xmas2, "Full.Phrase")

On the first day of Christmas, my true love gave to me:
an  email  from Cal Poly.

On the second day of Christmas, my true love gave to me:
two meal points, and
an  email  from Cal Poly.

On the third day of Christmas, my true love gave to me:
three lost pens,
two meal points, and
an  email  from Cal Poly.

On the fourth day of Christmas, my true love gave to me:
four course reviews,
three lost pens,
two meal points, and
an  email  from Cal Poly.

On the fifth day of Christmas, my true love gave to me:
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an  email  from Cal Poly.

On the sixth day of Christmas, my true love gave to me:
six  graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an  email  from Cal Poly.

On the seventh day of Christmas, my true love gave to me:
seven  seniors stressing,
six  graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an  email  fro